In [ ]:
#!/usr/bin/env python3
"""
Swing Monte Carlo Stock Screener (Yahoo Finance)

What it does:
- Downloads daily adjusted close for a list of tickers
- Estimates annualized drift (mu) and volatility (sigma) from lookback window
- Runs GBM Monte Carlo for a swing horizon (e.g., 10 trading days)
- Computes probability stock ends higher vs lower (P_up / P_down)
- Computes probability to TOUCH +Target% or -Target% within horizon
- Computes FIRST-PASSAGE odds: P(hit +Target before -Target) and vice versa
- Outputs a ranked CSV

Dependencies:
    pip install yfinance pandas numpy

Notes:
- This assumes a GBM model. Real returns may have fat tails/regime changes.
- Consider shrinking drift to reduce overfit (DRIFT_SHRINK in config).
- For NSE symbols use ".NS" suffix (e.g., INFY.NS).

Here’s what each column in your Monte Carlo swing screen means (all probabilities are 0–1, all returns are decimals—e.g., 0.022 = +2.2%):

ticker – The symbol you screened.

status – Whether the run succeeded (ok) or had an issue (no_data, param_error: …).

S0 – The latest price used as the simulation start (your “entry” reference).

mu_ann – Annualized drift estimate from recent daily log-returns (after shrinkage). Rough trend signal used inside the model (can be negative).

sigma_ann – Annualized volatility (std dev) from recent daily log-returns. Higher → wider price distribution. Approx daily vol ≈ sigma_ann / √252.

horizon_days – Number of trading days simulated (your swing horizon).

P_up – Probability terminal price after horizon_days ends above S0.

P_down – Probability terminal price ends below S0. (Rounding means P_up + P_down might not equal 1 exactly.)

P_touch_+5% – Probability price touches +5% (from S0) at any time during the horizon (intraperiod “barrier touch”, not necessarily the close).

P_touch_-5% – Probability price touches −5% sometime during the horizon.

P(+target first) – First-passage probability of hitting +5% before −5%. (Pathwise ordering; very useful for target/stop asymmetry.)

P(-target first) – First-passage probability of hitting −5% before +5%.

P(neither target) – Probability neither +5% nor −5% is touched before the horizon ends.

E[ret] – Expected terminal return (mean of S_T / S0 − 1) after horizon_days.

Median[ret] – Median terminal return (50th percentile).

Q05[ret] – 5th percentile terminal return (a “bad-case” tail; ~1-in-20 worse).

Q95[ret] – 95th percentile terminal return (a “good-case” tail; ~1-in-20 better).

How to read one line (example)

ETERNAL.NS

P_up = 0.612 → ~61.2% chance the stock finishes higher than 322.40 after 10 trading days.

P_touch_+5% = 0.48, P_touch_-5% = 0.273 → ~48% odds to tag +5% sometime vs ~27.3% to tag −5%.

P(+target first) = 0.471 vs P(-target first) = 0.267 → More likely to hit +5% before −5% than the opposite.

E[ret] = 0.0222 and Median[ret] = 0.0199 → Expected/median terminal gain ≈ +2.2% / +2.0% over 10 days.

Q05 = −0.0911, Q95 = +0.1425 → Middle 90% of outcomes roughly −9.1% to +14.2%.
"""

import math
import sys
from typing import Dict, Tuple
from datetime import datetime

import numpy as np
import pandas as pd
import yfinance as yf

# ==============================
# CONFIG — EDIT ME
# ==============================
TICKERS = ['360ONE.NS', '3MINDIA.NS', 'ABB.NS', 'ACC.NS', 'ACMESOLAR.NS', 'AIAENG.NS', 'APLAPOLLO.NS', 'AUBANK.NS', 'AWL.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABBOTINDIA.NS', 'ACE.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ATGL.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'AEGISLOG.NS', 'AFCONS.NS', 'AFFLE.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'APLLTD.NS', 'ALIVUS.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'ARE&M.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAZEN.NS', 'ASTRAL.NS', 'ATUL.NS', 'AUROPHARMA.NS', 'AIIL.NS', 'DMART.NS', 'AXISBANK.NS', 'BASF.NS', 'BEML.NS', 'BLS.NS', 'BSE.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BAJAJHLDNG.NS', 'BAJAJHFL.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'MAHABANK.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BERGEPAINT.NS', 'BDL.NS', 'BEL.NS', 'BHARATFORG.NS', 'BHEL.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BSOFT.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BBTC.NS', 'BOSCHLTD.NS', 'FIRSTCRY.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'MAPMYINDIA.NS', 'CCL.NS', 'CESC.NS', 'CGPOWER.NS', 'CRISIL.NS', 'CAMPUS.NS', 'CANFINHOME.NS', 'CANBK.NS', 'CAPLIPOINT.NS', 'CGCL.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CDSL.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAHLDNG.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'CUB.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CAMS.NS', 'CONCORDBIO.NS', 'CONCOR.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CROMPTON.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DCMSHRIRAM.NS', 'DLF.NS', 'DOMS.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'LALPATHLAB.NS', 'DRREDDY.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'EICHERMOT.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'NYKAA.NS', 'FEDERALBNK.NS', 'FACT.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FSL.NS', 'FIVESTAR.NS', 'FORTIS.NS', 'GAIL.NS', 'GVT&D.NS', 'GMRAIRPORT.NS', 'GRSE.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'MEDANTA.NS', 'GODIGIT.NS', 'GPIL.NS', 'GODFRYPHLP.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GESHIP.NS', 'FLUOROCHEM.NS', 'GUJGASLTD.NS', 'GMDCLTD.NS', 'GNFC.NS', 'GPPL.NS', 'GSPL.NS', 'HEG.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HFCL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HEROMOTOCO.NS', 'HSCL.NS', 'HINDALCO.NS', 'HAL.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'POWERINDIA.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDFCFIRSTB.NS', 'IFCI.NS', 'IIFL.NS', 'INOXINDIA.NS', 'IRB.NS', 'IRCON.NS', 'ITC.NS', 'ITI.NS', 'INDGN.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'IEX.NS', 'INDHOTEL.NS', 'IOC.NS', 'IOB.NS', 'IRCTC.NS', 'IRFC.NS', 'IREDA.NS', 'IGL.NS', 'INDUSTOWER.NS', 'INDUSINDBK.NS', 'NAUKRI.NS', 'INFY.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'INDIGO.NS', 'IGIL.NS', 'IKS.NS', 'IPCALAB.NS', 'JBCHEPHARM.NS', 'JKCEMENT.NS', 'JBMA.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JPPOWER.NS', 'J&KBANK.NS', 'JINDALSAW.NS', 'JSL.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JUSTDIAL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KPRMILL.NS', 'KEI.NS', 'KNRCON.NS', 'KPITTECH.NS', 'KAJARIACER.NS', 'KPIL.NS', 'KALYANKJIL.NS', 'KANSAINER.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KFINTECH.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KIMS.NS', 'LTF.NS', 'LTTS.NS', 'LICHSGFIN.NS', 'LTFOODS.NS', 'LTIM.NS', 'LT.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LUPIN.NS', 'MMTC.NS', 'MRF.NS', 'MGL.NS', 'MAHSEAMLES.NS', 'M&MFIN.NS', 'M&M.NS', 'MANAPPURAM.NS', 'MRPL.NS', 'MANKIND.NS', 'MARICO.NS', 'MARUTI.NS', 'MASTEK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'METROPOLIS.NS', 'MINDACORP.NS', 'MSUMI.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MCX.NS', 'MUTHOOTFIN.NS', 'NATCOPHARM.NS', 'NBCC.NS', 'NCC.NS', 'NHPC.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPCGREEN.NS', 'NTPC.NS', 'NH.NS', 'NATIONALUM.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NAM-INDIA.NS', 'NIVABUPA.NS', 'NUVAMA.NS', 'OBEROIRLTY.NS', 'ONGC.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'PAYTM.NS', 'OFSS.NS', 'POLICYBZR.NS', 'PCBL.NS', 'PGEL.NS', 'PIIND.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'PTCIL.NS', 'PVRINOX.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFIZER.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PEL.NS', 'PPLPHARMA.NS', 'POLYMED.NS', 'POLYCAB.NS', 'POONAWALLA.NS', 'PFC.NS', 'POWERGRID.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PNB.NS', 'RRKABEL.NS', 'RBLBANK.NS', 'RECLTD.NS', 'RHIM.NS', 'RITES.NS', 'RADICO.NS', 'RVNL.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RKFORGE.NS', 'RCF.NS', 'RTNINDIA.NS', 'RAYMONDLSL.NS', 'RAYMOND.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RPOWER.NS', 'ROUTE.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SRF.NS', 'SAGILITY.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'MOTHERSON.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'RENUKA.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'STARHEALTH.NS', 'SBIN.NS', 'SAIL.NS', 'SWSOLAR.NS', 'SUMICHEM.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANENERGY.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TBOTEK.NS', 'TVSMOTOR.NS', 'TANLA.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TTML.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'NIACL.NS', 'RAMCOCEM.NS', 'THERMAX.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TARIL.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRIVENI.NS', 'TRITURBINE.NS', 'TIINDIA.NS', 'UCOBANK.NS', 'UNOMINDA.NS', 'UPL.NS', 'UTIAMC.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UBL.NS', 'UNITDSPR.NS', 'USHAMART.NS', 'VGUARD.NS', 'DBREALTY.NS', 'VTL.NS', 'VBL.NS', 'MANYAVAR.NS', 'VEDL.NS', 'VIJAYA.NS', 'VMM.NS', 'IDEA.NS', 'VOLTAS.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZFCVINDIA.NS', 'ZEEL.NS', 'ZENTEC.NS', 'ZENSARTECH.NS', 'ZYDUSLIFE.NS', 'ECLERX.NS']


START_DATE     = "2010-01-01"
END_DATE       = None      # until today
LOOKBACK_DAYS  = 252 * 2   # use last ~2 years for parameter estimation
HORIZON_DAYS   = 10        # swing horizon in trading days
PATHS          = 20000     # MC paths per ticker
SEED           = 12345
DRIFT_SHRINK   = 0.5       # 0..1 (0 = ignore drift, 1 = full estimated drift)
TARGET_UP_PCT  = 0.05      # +5% target (touch probability & first-passage)
TARGET_DN_PCT  = 0.05      # -5% target

SAVE_CSV       = "swing_mc_screen.csv"
# ==============================

TRADING_DAYS = 252

def download_daily_adj_close(ticker: str, start: str, end: str):
    df = yf.download(
        ticker,
        start=start,
        end=end,
        auto_adjust=True,
        progress=False,
        threads=False
    )
    if isinstance(df, pd.DataFrame) and not df.empty:
        df = df.dropna()
        return df
    return pd.DataFrame()

def estimate_mu_sigma(prices: pd.Series, lookback_days: int) -> Tuple[float, float, float]:
    """
    Estimate annualized (mu, sigma) from the last lookback_days.
    Returns (S0, mu_ann, sigma_ann)
    """
    if len(prices) < 60:
        raise ValueError("Not enough price history (need >= 60 daily rows).")
    tail = prices.iloc[-lookback_days:] if len(prices) >= lookback_days else prices
    log_ret = np.log(tail).diff().dropna()
    mu_ann = float(log_ret.mean()) * TRADING_DAYS
    sigma_ann = float(log_ret.std(ddof=1)) * math.sqrt(TRADING_DAYS)
    S0 = float(tail.iloc[-1])
    # shrink drift to reduce noise
    mu_ann = DRIFT_SHRINK * mu_ann
    return S0, mu_ann, sigma_ann

def simulate_gbm(S0: float, mu_ann: float, sigma_ann: float, horizon_days: int, paths: int, seed: int) -> np.ndarray:
    """
    Simulate GBM paths at daily steps. Returns S with shape (steps+1, paths).
    """
    rng = np.random.default_rng(seed)
    dt = 1.0 / TRADING_DAYS
    steps = int(horizon_days)
    Z = rng.standard_normal((steps, paths))
    increments = (mu_ann - 0.5 * sigma_ann**2) * dt + sigma_ann * math.sqrt(dt) * Z
    log_cum = np.vstack([np.zeros(paths), np.cumsum(increments, axis=0)])
    S = S0 * np.exp(log_cum)
    return S

def first_cross_idx(S: np.ndarray, level: float, direction: str) -> np.ndarray:
    """
    Return the first index where S crosses 'level' in 'up' or 'down' direction.
    If never, returns steps+1 sentinel.
    """
    cond = (S >= level) if direction == "up" else (S <= level)
    cond[0, :] = False  # ignore t=0
    idx = cond.argmax(axis=0)
    never = ~cond.any(axis=0)
    idx[never] = S.shape[0]
    return idx

def evaluate_ticker(ticker: str) -> Dict[str, float]:
    df = download_daily_adj_close(ticker, START_DATE, END_DATE)
    if df.empty or "Close" not in df.columns:
        return {"ticker": ticker, "status": "no_data"}

    prices = df["Close"].astype(float)
    try:
        S0, mu_ann, sigma_ann = estimate_mu_sigma(prices, LOOKBACK_DAYS)
    except Exception as e:
        return {"ticker": ticker, "status": f"param_error: {e}"}

    S = simulate_gbm(S0, mu_ann, sigma_ann, HORIZON_DAYS, PATHS, SEED)

    S_T = S[-1, :]
    ret_T = S_T / S0 - 1.0

    # Up/Down probabilities at horizon
    p_up = float(np.mean(S_T > S0))
    p_down = float(np.mean(S_T < S0))

    # Touch probabilities (ever touch within horizon)
    up_level = S0 * (1.0 + TARGET_UP_PCT)
    dn_level = S0 * (1.0 - TARGET_DN_PCT)

    touch_up = (S >= up_level).any(axis=0)
    touch_dn = (S <= dn_level).any(axis=0)
    p_touch_up = float(np.mean(touch_up))
    p_touch_dn = float(np.mean(touch_dn))

    # First-passage odds
    idx_up = first_cross_idx(S, up_level, "up")
    idx_dn = first_cross_idx(S, dn_level, "down")
    steps = S.shape[0] - 1
    up_first = idx_up < idx_dn
    dn_first = idx_dn < idx_up
    none_first = (idx_up > steps) & (idx_dn > steps)
    p_up_before_dn = float(np.mean(up_first))
    p_dn_before_up = float(np.mean(dn_first))
    p_neither = float(np.mean(none_first))

    # Summary stats of return distribution
    exp_ret = float(np.mean(ret_T))
    median_ret = float(np.median(ret_T))
    q05 = float(np.percentile(ret_T, 5))
    q95 = float(np.percentile(ret_T, 95))

    out = {
        "ticker": ticker,
        "status": "ok",
        "S0": S0,
        "mu_ann": mu_ann,
        "sigma_ann": sigma_ann,
        "horizon_days": HORIZON_DAYS,
        "P_up": p_up,
        "P_down": p_down,
        "P_touch_+{:.0f}%".format(TARGET_UP_PCT*100): p_touch_up,
        "P_touch_-{:.0f}%".format(TARGET_DN_PCT*100): p_touch_dn,
        "P(+target first)": p_up_before_dn,
        "P(-target first)": p_dn_before_up,
        "P(neither target)": p_neither,
        "E[ret]": exp_ret,
        "Median[ret]": median_ret,
        "Q05[ret]": q05,
        "Q95[ret]": q95,
    }
    return out

def main():
    rows = []
    print(f"Running swing MC screen on {len(TICKERS)} tickers | horizon={HORIZON_DAYS}d paths={PATHS}")
    for t in TICKERS:
        print(f"  > {t}")
        res = evaluate_ticker(t)
        rows.append(res)

    df = pd.DataFrame(rows)
    if df.empty:
        print("No results.")
        return

    # Order good-to-bad (higher P_up, then higher E[ret], then lower sigma)
    by_cols = ["P_up", "E[ret]", "sigma_ann"]
    for c in [c for c in by_cols if c not in df.columns]:
        df[c] = np.nan

    df = df.sort_values(by=["status"] + by_cols, ascending=[True, False, False, True])

    # Round pretty
    prob_cols = [c for c in df.columns if c.startswith("P")]
    df[prob_cols] = df[prob_cols].astype(float).round(3)
    df["E[ret]"] = df["E[ret]"].astype(float).round(4)
    df["Median[ret]"] = df["Median[ret]"].astype(float).round(4)
    for col in ["mu_ann", "sigma_ann"]:
        df[col] = df[col].astype(float).round(4)

    print("\n=== Swing MC Screen ===")
    print(df.to_string(index=False))

    if SAVE_CSV:
        df.to_csv(SAVE_CSV, index=False)
        print(f"\nSaved: {SAVE_CSV}")

if __name__ == "__main__":
    main()


Running swing MC screen on 500 tickers | horizon=10d paths=20000
  > 360ONE.NS


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:76: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mu_ann = float(log_ret.mean()) * TRADING_DAYS
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sigma_ann = float(log_ret.std(ddof=1)) * math.sqrt(TRADING_DAYS)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S0 = float(tail.iloc[-1])


  > 3MINDIA.NS


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:76: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mu_ann = float(log_ret.mean()) * TRADING_DAYS
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sigma_ann = float(log_ret.std(ddof=1)) * math.sqrt(TRADING_DAYS)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S0 = float(tail.iloc[-1])


  > ABB.NS


...

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:76: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mu_ann = float(log_ret.mean()) * TRADING_DAYS
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sigma_ann = float(log_ret.std(ddof=1)) * math.sqrt(TRADING_DAYS)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S0 = float(tail.iloc[-1])


  > ZYDUSLIFE.NS


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:76: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mu_ann = float(log_ret.mean()) * TRADING_DAYS
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sigma_ann = float(log_ret.std(ddof=1)) * math.sqrt(TRADING_DAYS)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S0 = float(tail.iloc[-1])


  > ECLERX.NS

=== Swing MC Screen ===
       ticker status            S0  mu_ann  sigma_ann  horizon_days  P_up  P_down  P_touch_+5%  P_touch_-5%  P(+target first)  P(-target first)  P(neither target)  E[ret]  Median[ret]  Q05[ret]  Q95[ret]
   ETERNAL.NS     ok    322.399994  0.5424     0.3493            10 0.612   0.388        0.480        0.273             0.471             0.267              0.262  0.0222       0.0199 -0.091094  0.142479
     GVT&D.NS     ok   2752.899902  0.5166     0.4504            10 0.578   0.422        0.538        0.388             0.507             0.363              0.130  0.0213       0.0174 -0.123057  0.177738
ANANDRATHI.NS     ok   2936.699951  0.3633     0.3380            10 0.577   0.423        0.434        0.290             0.428             0.285              0.287  0.0149       0.0128 -0.094030  0.130377
BHARTIARTL.NS     ok   1887.199951  0.2014     0.2157            10 0.570   0.430        0.239        0.137             0.239             0.137  

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:76: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mu_ann = float(log_ret.mean()) * TRADING_DAYS
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sigma_ann = float(log_ret.std(ddof=1)) * math.sqrt(TRADING_DAYS)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_61094/4004665421.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S0 = float(tail.iloc[-1])
